In [7]:
from refiningEventLabels.lib.graph.graphTool import graphTool 
from refiningEventLabels.lib.objects.customParameters import customParameters
from refiningEventLabels.lib.refinement.labelRefinement import verticalRefinement, horizontalRefinement
from refiningEventLabels.lib.eventLogProcessing.DBTool import DBTool 
from refiningEventLabels.lib.eventLogProcessing.postProcessing import eventLogRenaming
from refiningEventLabels.lib.costFunction.mappings import createEventIDs, positionsOfCandidates
from refiningEventLabels.lib.costFunction.cost import bestMappings
from pm4py.objects.log.importer.xes import factory as xes_import_factory
import numpy as np

#REPLACE with data from local web server
log = xes_import_factory.apply("running-example.xes")
orgLog = xes_import_factory.apply("running-example.xes")
cp = customParameters(candidateLabels = ["decide","examine casually"],
                      horizontalThreshold = 0.2,
                      verticalThreshold = 0.2, 
                      weightStructure = 0.3, 
                      weightMatch = 0.3, 
                      weightNoMatch = 0.3)


#PreProcessing Step
db = DBTool(log)
variants = createEventIDs(db.getVariants())
count = len(variants) 
C = np.zeros((count,count)) 
  
#Computing best mappings and MAX(cost)
best_mappings = bestMappings(cp, variants, C)
max_cost = np.amax(C)
C = C/max_cost

#Intermediate steps before creating the graph
candidates = cp.getCandidateLabels()
pos_candidates = positionsOfCandidates(candidates, variants)

#Graph creation 
G = graphTool()
G.createGraphFromVariants(variants)
G.addOptimalMappings(best_mappings, max_cost, pos_candidates)

#Refinement Steps
subgraphs = G.clusterDetection(cp)
horizontalRefinement(cp, subgraphs)
verticalRefinement(cp, subgraphs, db)

#PostProcessing Step
eventLogRenaming(cp, subgraphs, db, log)

[{'attributes': {'concept:name': '3', 'creator': 'Fluxicon Nitro'}, 'events': [{'concept:name': 'register request', 'org:resource': 'Pete', 'time:timestamp': datetime.datetime(2010, 12, 30, 14, 32, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'Activity': 'register request', 'Resource': 'Pete', 'Costs': '50'}, '..', {'concept:name': 'pay compensation', 'org:resource': 'Ellen', 'time:timestamp': datetime.datetime(2011, 1, 15, 10, 45, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'Activity': 'pay compensation', 'Resource': 'Ellen', 'Costs': '200'}]}, '....', {'attributes': {'concept:name': '4', 'creator': 'Fluxicon Nitro'}, 'events': [{'concept:name': 'register request', 'org:resource': 'Pete', 'time:timestamp': datetime.datetime(2011, 1, 6, 15, 2, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'Activity': 'register request', 'Resource': 'Pete', 'Costs': '50'}, '..', {'concept:name': 'reject request', 'org:resource': 'Ellen', 'time:timestamp': 

In [8]:
for case_index, case in enumerate(orgLog):
    print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
    for event_index, event in enumerate(case):
        print("event index: %d  event activity: %s" % (event_index, event["concept:name"]))


 case index: 0  case id: 3
event index: 0  event activity: register request
event index: 1  event activity: examine casually
event index: 2  event activity: check ticket
event index: 3  event activity: decide
event index: 4  event activity: reinitiate request
event index: 5  event activity: examine thoroughly
event index: 6  event activity: check ticket
event index: 7  event activity: decide
event index: 8  event activity: pay compensation

 case index: 1  case id: 2
event index: 0  event activity: register request
event index: 1  event activity: check ticket
event index: 2  event activity: examine casually
event index: 3  event activity: decide
event index: 4  event activity: pay compensation

 case index: 2  case id: 1
event index: 0  event activity: register request
event index: 1  event activity: examine thoroughly
event index: 2  event activity: check ticket
event index: 3  event activity: decide
event index: 4  event activity: reject request

 case index: 3  case id: 6
event ind

In [9]:
for case_index, case in enumerate(log):
    print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
    for event_index, event in enumerate(case):
        print("event index: %d  event activity: %s" % (event_index, event["concept:name"]))


 case index: 0  case id: 3
event index: 0  event activity: register request
event index: 1  event activity: examine casually1.1
event index: 2  event activity: check ticket
event index: 3  event activity: decide1.1
event index: 4  event activity: reinitiate request
event index: 5  event activity: examine thoroughly
event index: 6  event activity: check ticket
event index: 7  event activity: decide1.2
event index: 8  event activity: pay compensation

 case index: 1  case id: 2
event index: 0  event activity: register request
event index: 1  event activity: check ticket
event index: 2  event activity: examine casually2.1
event index: 3  event activity: decide2.1
event index: 4  event activity: pay compensation

 case index: 2  case id: 1
event index: 0  event activity: register request
event index: 1  event activity: examine thoroughly
event index: 2  event activity: check ticket
event index: 3  event activity: decide3.1
event index: 4  event activity: reject request

 case index: 3  ca